## Linear trees using LightGBM for forecasting

* Timeseries with trend
    * Tree based models cannot extrapolate 
        * Break the timeseries in to it's components
            * Forcaste the trend seperately and add it back
        * Advanced tree algorithm
            * Linear trees : fits a linear model at the leaf 

#### OPTION:1
* $Estimate \: the \: trend \: \: T_t = \beta_0 + \beta_1t + \beta_2t^2 + ...$
    * We can use $PolynomialTrendForecaster$ from sktime for estimating trend
* $Detrend \: the \: target \: variable \: Y_{detrend} = Y_t - T_t$
* $Build \: a \: forecaster \: on \: detrended \: data \: \: Z_{forecast} = Tree(Y_{detrend})$
* $Forecast \: the \: trend \: using \: any \: other \: method \:\: T_{forecast} = \beta_0 + \beta_1t + \beta_2t^2 + ...$
* $Add \: the \: trend \: forecast \: to \: the \: detrended \: data \: forecast \: \: Y_{forecast} =  Z_{forecast} + T_{forecast}$

#### OPTION : 2
* Use Advanced tree algorithms like LGBMRegressor that fits a complex model in the leaf nodes
    $$model = LGBMRegressor(linear\_tree=True)$$


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformations.series.boxcox import BoxCoxTransformer, LogTransformer
from sktime.transformations.series.summarize import WindowSummarizer
from sktime.transformations.series.detrend import Detrender
from sktime.transformations.series.time_since import TimeSince

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline, make_union
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

from lightgbm import LGBMRegressor

#### The air passengers dataset is the monthly totals of international airline passengers, from 1949 to 1960, in units of 1000s. 

In [2]:
data = pd.read_csv("../../Datasets/example_air_passengers.csv", parse_dates=['ds'], index_col=['ds'])

data.plot(figsize=(15,4))

<img src='./plots/air-passengers-data.png'>

## Preprocess

In [22]:
time_since = TimeSince(keep_original_columns=False, freq='MS')
polynomial = PolynomialFeatures(degree=3, interaction_only=False)

time_features = make_pipeline(time_since, polynomial)


window_summary = WindowSummarizer(
    lag_feature={
    'lag' : [1,2,3,12],
    'mean' : [[1,12]]
    },
    truncate='bfill',
    target_cols=['y'],
    
)

feature_transformer = make_union(time_features, window_summary)

feature_transformer_scaled = make_pipeline(feature_transformer, MinMaxScaler())

## Train Test Split

In [23]:
forecast_start_time = pd.to_datetime("1958-01-01")
df_train = data.query(f'index < "{forecast_start_time}"').asfreq('MS')
df_test = data.query(f'index >= "{forecast_start_time}"').asfreq('MS')

### Train the model

In [26]:
model = LGBMRegressor(boosting_type='gbdt', linear_tree=True, reg_lambda=0.1, n_estimators=100)

X = feature_transformer_scaled.fit_transform(df_train)
y = df_train['y']

model.fit(X,y)

train_preds = model.predict(X)
train_preds = pd.DataFrame(data=train_preds, index=df_train.index, columns=['y'])

In [30]:
ax=df_train.plot(figsize=(15,4))
train_preds.plot(ax=ax)
ax.legend(['train-data','train_predictions'])

<img src='./plots/air-passengers-data-train-set-predictions-lgbdt.png'>

### Recursive forecast

In [37]:
forecast_horizon = pd.date_range(start=forecast_start_time, periods=len(df_test), freq='MS')

# we need 12 months data in the past to create lag features
lookback_window_size = pd.DateOffset(months=12)

lookback_df = data.query(f'index >= "{forecast_start_time - lookback_window_size}"').asfreq('MS')

y_test_pred = []
for fh in forecast_horizon:
    X = lookback_df.loc[:fh]
    X = feature_transformer_scaled.transform(X)

    y_pred = model.predict([X[-1]])
    y_test_pred.append(y_pred)


In [44]:
y_test_pred_df = pd.DataFrame(data=y_test_pred, index=df_test.index, columns=['y_test_pred'])

ax = df_test.plot(figsize=(15,4), marker='.')
y_test_pred_df.plot(ax=ax, marker='.')
ax.legend(['test-data', 'test-data-predictions']);

<img src='./plots/air-passengers-data-test-data-prediciton-lgbdt.png'>

In [46]:
mae = mean_absolute_error(y_true=df_test['y'], y_pred=y_test_pred_df['y_test_pred'])
rmse = mean_squared_error(y_true=df_test['y'], y_pred=y_test_pred_df['y_test_pred'], squared=False)
mape = mean_absolute_percentage_error(y_true=df_test['y'], y_pred=y_test_pred_df['y_test_pred'])

In [49]:

ax=df_train.plot(figsize=(15,4))
train_preds.plot(ax=ax)

df_test.plot(ax=ax, marker='.')
y_test_pred_df.plot(ax=ax, marker='.')

ax.set(title=f'Forecasting | MSE :{mae :0.2f} | RMSE : {rmse: 0.2f} | MAPE : {mape :0.2f}')
ax.legend(['train-data','train_predictions', 'test-data', 'test-data-predictions']);

<img src='./plots/air-passengers-data-full-forecast-lgbdt.png'>